In [ ]:
%%capture

! pip install datasets
! pip install transformers
! pip install accelerate -U

# Скачивание и обработка данных

In [ ]:
from datasets import load_dataset

dataset = load_dataset('conll2003') # скачаем английский датасет с конференции, исходник: https://huggingface.co/datasets/conll2003
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
# https://www.ibm.com/docs/en/wca/3.5.0?topic=analytics-part-speech-tag-sets

tags = dataset['train'].features['pos_tags'].feature # посмотрим как выглядят наши тэги
tags

ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None)

In [ ]:
# создадим словарь, которые будет соотносить слова с числом (для pos)
# наши данные для pos изначально представлены числом, поэтому сначала создадим id2label
tags = tags.names
id2label = {name:encoded for name, encoded in enumerate(tags)}
label2id = {v:k for k, v in id2label.items()}

In [ ]:
# посмотрим на наши данные
print(dataset["train"][0]["tokens"])
print(dataset["train"][0]["pos_tags"])
print([id2label[name] for name in dataset["train"][0]["pos_tags"]])

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
[22, 42, 16, 21, 35, 37, 16, 21, 7]
['NNP', 'VBZ', 'JJ', 'NN', 'TO', 'VB', 'JJ', 'NN', '.']


In [ ]:
from transformers import AutoTokenizer

# pos уже представлены в виде числа, однако слова -- нет
# для этого нам нужно токенизатор, мы воспользуемся bert'ом
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
# посмотрим как он работает
tokenized_first_sentence = tokenizer(dataset["train"][0]["tokens"], is_split_into_words=True)
print(tokenized_first_sentence.tokens())
print(tokenized_first_sentence.word_ids())

['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]']
[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]


In [ ]:
# видим, что токенизатор добавил свои специальные токены
# однако у наших тэгов их нет, то есть длина векторизированных токенов и pos'ов
# кроме того bert используется wordpiece алгоритм, это значит что он  делит незнакомые слова на части (как в примере с lamb)
# в итоге вектор наших тэггв и слова будет отличаться,
# так быть не должно, исправим

def align_labels(labels, words):
  new_labels = []
  current_word = None
  for word_id in words:
      if word_id != current_word:
          current_word = word_id
          label = -100 if word_id is None else labels[word_id] # -100 - это наш филлер
          new_labels.append(label)
      elif word_id is None:
          new_labels.append(-100)
      else:
          label = labels[word_id]
          if label % 2 == 1:
              label += 1
          new_labels.append(label)
  return new_labels

In [ ]:
# посмотрим что получилось
labels = dataset["train"][0]["pos_tags"]
print('До: ', labels)
aligned = align_labels(labels,
                   tokenized_first_sentence.word_ids())
print('После: ', aligned)
print(f'Одинаковы ли по размеру слова и тэги? --> {len(aligned) == len(tokenized_first_sentence.word_ids())}')

До:  [22, 42, 16, 21, 35, 37, 16, 21, 7]
После:  [-100, 22, 42, 16, 21, 35, 37, 16, 21, 22, 7, -100]
Одинаковы ли по размеру слова и тэги? --> True


In [ ]:
def tokenize_and_align(data):
    tokenized_inputs = tokenizer(
        data["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = data["pos_tags"] # нас интересует только pos
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
# векторизуем весь наш датасет
vectorized_dataset = dataset.map(
    tokenize_and_align,
    batched=True,
    batch_size=8,
    remove_columns=dataset['train'].column_names # колонки у всех одинаковые
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
vectorized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

# Обучение

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint,
                                                        id2label=id2label,
                                                        label2id=label2id)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=vectorized_dataset["train"],
    eval_dataset=vectorized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.653700,0.313206
2,0.197100,0.283404
3,0.147400,0.273638
4,0.110500,0.271139
5,0.089100,0.301838
6,0.067900,0.300723
7,0.054400,0.319670
8,0.046100,0.326005


TrainOutput(global_step=7024, training_loss=0.1492057879856344, metrics={'train_runtime': 1480.1335, 'train_samples_per_second': 75.89, 'train_steps_per_second': 4.746, 'total_flos': 2808193374304542.0, 'train_loss': 0.1492057879856344, 'epoch': 8.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.3260047733783722,
 'eval_runtime': 11.7174,
 'eval_samples_per_second': 277.365,
 'eval_steps_per_second': 17.41,
 'epoch': 8.0}

In [ ]:
trainer.save_model('./bert-pos-tagger') #сохраним модель

# Использование

In [ ]:
# чтобы убедиться в эффекивности нашей модели
# сравним ее результат с обычным пос-тэггером из nltk

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
from transformers import pipeline

model_checkpoint = "./bert-pos-tagger"
nlp = pipeline(
    "token-classification",
    model=model_checkpoint,
    aggregation_strategy="simple"
)

In [ ]:
# возьмем такое предложение

amb_sentence = "I would like to book the book"
# здесь первый book - это глагол, а второй -- существительное,
# об этом наш говорят предыдущие слова: to используется с глаголом, the - c существительным
# простой тэггер не сможет с этим справиться, а вот модель, которая смотрит на контект — сможет

In [ ]:
pos_tag(word_tokenize(amb_sentence)) # оба слова book - были распознаны как сущ

[('I', 'PRP'),
 ('would', 'MD'),
 ('like', 'VB'),
 ('to', 'TO'),
 ('book', 'NN'),
 ('the', 'DT'),
 ('book', 'NN')]

In [ ]:
nlp(amb_sentence) # видим, что наша модель успешно сняла омонимию со слова book

[{'entity_group': 'PRP', 'score': 0.999246, 'word': 'I', 'start': 0, 'end': 1},
 {'entity_group': 'MD',
  'score': 0.9991141,
  'word': 'would',
  'start': 2,
  'end': 7},
 {'entity_group': 'VB',
  'score': 0.99644804,
  'word': 'like',
  'start': 8,
  'end': 12},
 {'entity_group': 'TO',
  'score': 0.99955803,
  'word': 'to',
  'start': 13,
  'end': 15},
 {'entity_group': 'VB',
  'score': 0.99891186,
  'word': 'book',
  'start': 16,
  'end': 20},
 {'entity_group': 'DT',
  'score': 0.99981076,
  'word': 'the',
  'start': 21,
  'end': 24},
 {'entity_group': 'NN',
  'score': 0.999757,
  'word': 'book',
  'start': 25,
  'end': 29}]